In [1]:
import numpy as np

In [2]:
generation_1 = np.random.normal(loc=100, scale=15, size=100000)

In [209]:
def get_married(previous_generation: np.array, correlation_coefficient: float = 0.9) -> np.array:
    """
    pairs up values in the previous generation so that their IQs have the corresponding correlation coefficient.
    """
    n_pairs = previous_generation.shape[0] // 2

    men = previous_generation[:n_pairs]
    women = previous_generation[n_pairs:]

    standardized_men = (men - 100) / 15
    standardized_women = (women - 100) / 15
    noise = np.random.normal(loc=0, scale=1, size=n_pairs)
    correlated_woman_values = (correlation_coefficient * standardized_men) + (np.sqrt(1 - correlation_coefficient ** 2) * noise)

    pairs = np.array((standardized_women.shape[0], 2))
    standardized_women_to_original_mapping = {standardized_woman : original_woman for standardized_woman, original_woman in zip(standardized_women, women)}
    for idx, val in enumerate(list(correlated_woman_values)):
        woman_idx = np.abs(standardized_women - val).argmin()
        standardized_woman_value = standardized_women[int(woman_idx)]
        woman_value = standardized_women_to_original_mapping[standardized_woman_value]
        pairs = np.vstack([pairs, [men[idx], woman_value]])
        standardized_women = np.delete(standardized_women, woman_idx)

    return pairs[1:]


def denormalize_array(array: np.array) -> np.array:
    return array * 15 + 100

    return array

def have_children(parents: np.array) -> np.array:
    """
    For simplicity's sake, assumes all parents have two children, keeping population size constant.
    """
    parent_mean_iq = parents.mean(axis=1).flatten()
    parent_normalized = (parent_mean_iq - 100) / 15
    correlation_parent_child = 0.40
    noise_1 = np.random.normal(loc=0, scale=1, size=parents.shape[0])
    noise_2 = np.random.normal(loc=0, scale=1, size=parents.shape[0])
    child_1 = (correlation_parent_child * parent_normalized) + (np.sqrt(1 - correlation_parent_child**2) * noise_1)
    child_2 = (correlation_parent_child * parent_normalized) + (np.sqrt(1 - correlation_parent_child**2) * noise_2)
    normalized_children = np.column_stack((child_1, child_2))
    scaled_children = denormalize_array(normalized_children)
    return scaled_children


def evaluate_percentiles(people_pairs, low_percentile: int = 5, high_percentile: int = 95) -> np.array:
    return np.percentile(np.vstack((people_pairs[:, 0], people_pairs[:, 1])), [low_percentile, high_percentile])


def simulate_n_generations(n: int, unmarried_parents: np.array) -> np.array:
    print(f"{n} generations remaining.")
    couples = get_married(unmarried_parents)
    children = have_children(couples)
    if n > 1:
        return simulate_n_generations(n-1, children.flatten())
    else:
        return children

# Verify on a Single Generation

In [144]:
couples = get_married(generation_1, 0.4)

In [145]:
correlation = np.corrcoef(couples[:, 0], couples[:, 1])[0, 1]
correlation

np.float64(0.3971439216894434)

In [151]:
children = have_children(couples)

[-0.50717041 -1.27462081 -0.21848656 ...  1.12091456  1.65592223
  2.21955546]


In [152]:
np.corrcoef(children[:, 1], np.mean([couples[:, 0], couples[:, 1]], axis=0))[0,1]

np.float64(0.34143698258354566)

In [165]:
evaluate_percentiles(couples)

array([ 75.28144157, 124.6077048 ])

In [166]:
evaluate_percentiles(children)

array([ 76.07526055, 124.13508908])

# Run Simulation Across Multiple Generations

In [210]:
num_generations = 10
children = simulate_n_generations(num_generations, generation_1)

10 generations remaining.
9 generations remaining.
8 generations remaining.
7 generations remaining.
6 generations remaining.
5 generations remaining.
4 generations remaining.
3 generations remaining.
2 generations remaining.
1 generations remaining.


In [211]:
evaluate_percentiles(children, 1, 99)

array([ 64.99387539, 134.74333516])

In [212]:
evaluate_percentiles(couples, 1, 99)

array([ 64.9561008 , 134.93714092])